In [1]:
import os
import cv2
import json
import pathlib
import numpy as np
import pandas as pd
import tensorflow as tf

import matplotlib.pyplot as plt

from PIL import Image
from datetime import datetime as dt

os: The os library, as introduced earlier, allows us to interact with the operating system. It helps us manage file paths, directories, and other system-related operations efficiently.

cv2: OpenCV (cv2) is a powerful library for computer vision and image processing tasks. Its inclusion in our program signifies our exploration into visual data analysis and manipulation.

json: JSON (JavaScript Object Notation) is a lightweight data interchange format. By incorporating the json library, we are equipped to handle data in JSON format and perform related operations.

pathlib: As discussed previously, pathlib offers a more object-oriented approach to handle file paths and directories, providing a robust and platform-independent solution.

numpy (np): NumPy is the fundamental package for scientific computing with Python. It enables efficient numerical computations, particularly with arrays and matrices.

pandas (pd): Pandas, already introduced, plays a central role in data manipulation, organization, and analysis. Its versatile DataFrames help us process and explore data with ease.

tensorflow (tf): TensorFlow is an open-source machine learning framework that enables us to build and train deep learning models, opening the door to powerful predictive and analytical capabilities.

matplotlib.pyplot (plt): Matplotlib is a comprehensive library for creating static, interactive, and animated visualizations in Python. The pyplot module enables us to generate a wide range of plots and charts to visualize our data effectively.

PIL (Image): The Python Imaging Library (PIL) provides image processing capabilities, enabling us to work with images in various formats.

datetime (dt): The datetime module allows us to work with date and time data efficiently, which is particularly useful when dealing with temporal aspects in our ECG record analysis.

By incorporating these additional libraries, we expand the scope of our Python program to encompass image processing, machine learning, JSON data handling, and advanced visualization. As we progress further, these powerful tools will enhance our ability to draw deeper insights from the ECG records and enable us to present our findings in visually compelling ways.

In [2]:
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras.applications import mobilenet_v3
from tensorflow.keras.layers import Dense,GlobalMaxPooling2D
from tensorflow.keras.optimizers import RMSprop, Adadelta
from tensorflow.keras.preprocessing.image import ImageDataGenerator

tensorflow.keras: TensorFlow 2.0 introduced the Keras API as the official high-level API for deep learning. The integration of tensorflow.keras enables us to access and utilize various pre-built models, layers, optimizers, and data augmentation utilities seamlessly.

Model: The Model class from tensorflow.keras allows us to build our custom neural network models, combining layers and creating complex architectures to suit our specific requirements.

layers: TensorFlow and Keras provide a wide range of built-in layers for building neural networks. We can use layers like convolutional, pooling, dense, and more to construct sophisticated models.

mobilenet_v3: MobileNetV3 is a popular pre-trained model for image classification tasks. By utilizing mobilenet_v3, we can leverage its powerful features to perform image-based tasks effectively.

Dense and GlobalMaxPooling2D: These are specific layers from Keras that play a significant role in defining the structure and functionality of our neural network models.

RMSprop and Adadelta: These are optimization algorithms that can be used to train our deep learning models efficiently.

ImageDataGenerator: The ImageDataGenerator class simplifies the process of loading, preprocessing, and augmenting image data during model training. It enhances our ability to work with image datasets and achieve better model generalization.

By integrating TensorFlow and Keras into our Python program, we lay the foundation for powerful deep learning models that can discern patterns, classify images, and even aid in predicting heart-related conditions based on ECG records.

In [3]:
from tensorflow.python.client import device_lib
tf.test.is_built_with_cuda()
print("Num GPUs Available: ", len(device_lib.list_local_devices()))

Num GPUs Available:  1


from tensorflow.python.client import device_lib: This import allows us to access the device_lib module, which provides information about the available devices, including GPUs.

tf.test.is_built_with_cuda(): The is_built_with_cuda() function checks if TensorFlow was built with CUDA support, which is essential for GPU acceleration. CUDA is a parallel computing platform and programming model developed by NVIDIA for general-purpose computing on GPUs.

device_lib.list_local_devices(): The list_local_devices() function returns a list of all local devices available for computation, including both CPUs and GPUs.

Checking for GPU Availability:

The first line of code checks if TensorFlow was built with CUDA support by calling tf.test.is_built_with_cuda(). This ensures that our TensorFlow installation is GPU-enabled, allowing us to take advantage of GPU acceleration.

The second line, print("Num GPUs Available: ", len(device_lib.list_local_devices())), reports the number of GPUs available on the system. It does this by calling device_lib.list_local_devices() to get a list of all local devices, and then the length of this list corresponds to the number of GPUs available.

The Significance of GPU Acceleration:
When working with deep learning models, especially complex neural networks with large datasets, training can be computationally intensive. GPUs excel at parallel processing, making them highly effective for accelerating matrix operations and model training. Utilizing GPUs allows us to perform thousands of computations simultaneously, dramatically reducing the time required for model training and making it feasible to train complex models that would be infeasible on CPUs alone.The availability of GPUs empowers us to take full advantage of the hardware resources and supercharge our deep learning model training. 

In [4]:
IMG_SIZE = 224
batch_size=100

IMG_SIZE: The IMG_SIZE parameter represents the desired image size used during data preprocessing and model training. It defines the width and height of the image in pixels. In this case, the IMG_SIZE is set to 224.

batch_size: The batch_size parameter determines the number of samples processed in each training iteration. During model training, the data is divided into smaller batches, and the model updates its parameters based on the gradients computed for each batch. Setting an appropriate batch_size is crucial for optimizing memory usage and ensuring efficient utilization of hardware resources during training.

The Impact of Image Size and Batch Size:

Image Size (IMG_SIZE): The choice of IMG_SIZE can significantly impact both the model's performance and computational requirements. Smaller image sizes generally lead to faster training times but may sacrifice some fine-grained details in the images. Conversely, larger image sizes provide more detail to the model but might require more memory and computation during training.

A common practice is to use power-of-two image sizes (e.g., 224, 256, 512) as they are more computationally efficient for modern deep learning frameworks and GPU accelerators.
Batch Size: The batch_size parameter is crucial for optimizing model training. Choosing a batch size that is too large may lead to memory limitations, especially when working with GPUs that have limited memory capacity. Conversely, a batch size that is too small might lead to slower convergence and less efficient model training.

A typical approach is to experiment with different batch sizes, starting with moderate values (e.g., 32, 64, 128) and adjusting as needed based on memory constraints and training performance.
Optimizing the Hyperparameters:
To optimize IMG_SIZE and batch_size for our ECG record image analysis, we need to consider factors such as available hardware resources (e.g., GPU memory), the size of the dataset, and the complexity of the deep learning model.

It's advisable to experiment with different IMG_SIZE values and observe how they impact model performance and memory requirements.

For batch_size, we can start with moderate values and gradually increase or decrease them based on how the model performs during training. Additionally, consider the trade-off between larger batch sizes (more efficient but memory-intensive) and smaller batch sizes (less efficient but memory-friendly).

By carefully selecting the optimal values for IMG_SIZE and batch_size, we can strike a balance between model performance, training efficiency, and hardware utilization. These hyperparameters form a crucial part of our deep learning journey, guiding us towards achieving the best results in our ECG record analysis.We fine-tune our hyperparameters, uncover patterns in the ECG data, and witness the transformative power of deep learning in healthcare data analysis.

In [5]:
%%time
ds_train=tf.keras.preprocessing.image_dataset_from_directory("D:\\ECG DB\\FourClassesWithNSRmixed1519",
                                                             labels='inferred',label_mode="int",
                                                             class_names=['AF','NSR','PAC','PVC'],color_mode='rgb',
                                                             batch_size=batch_size,image_size=(IMG_SIZE,IMG_SIZE), #reshapeauto
                                                             shuffle=True,seed=123,validation_split=0.4,subset="training")

Found 2421 files belonging to 4 classes.
Using 1453 files for training.
CPU times: total: 297 ms
Wall time: 327 ms


In our Python program, we have introduced the tf.keras.preprocessing.image_dataset_from_directory() function to prepare our image dataset for deep learning. Profiling the execution time of this function using the %%time magic command allows us to gain insights into the time taken to preprocess the dataset.

%%time: The %%time magic command is a Jupyter Notebook feature that allows us to measure the execution time of a cell. When we execute the cell containing this magic command, it will report the time taken for the cell's execution.

tf.keras.preprocessing.image_dataset_from_directory(): This function is a powerful utility provided by TensorFlow that automatically creates a labeled dataset from a directory containing image files. It performs data preprocessing, label encoding, and batching, all in one step, streamlining the process of preparing image data for model training.

Dataset Preprocessing Parameters:

"D:\\ECG DB\\FourClassesWithNSRmixed1519": The path to the directory containing our image dataset.

labels='inferred': Inferred labels are derived from the directory structure, where each subdirectory represents a different class, and the subdirectory names are used as class labels.

label_mode="int": The labels are encoded as integers, corresponding to the class index.

class_names=['AF', 'NSR', 'PAC', 'PVC']: The names of the classes present in the dataset.

color_mode='rgb': The color mode of the images (RGB format).

batch_size: The batch size used during training.

image_size=(IMG_SIZE, IMG_SIZE): The desired size of the images for preprocessing.

shuffle=True: Shuffles the data to introduce randomness during training.

seed=123: The random seed used for shuffling the data.

validation_split=0.4: Splits the dataset, reserving 40% for validation, and 60% for training.

subset="training": Specifies that we are creating a dataset for training purposes.

Profiling the Execution Time:
By using %%time before executing the cell, we can monitor the execution time of this particular dataset preprocessing step. The output will show us the time taken for the entire preprocessing process.

The profiling information allows us to assess the efficiency of dataset preparation and gives us valuable insights into how much time this step takes. It helps us optimize our workflow and identify potential bottlenecks in our data pipeline.

In [6]:
%%time
ds_validate=tf.keras.preprocessing.image_dataset_from_directory("D:\\ECG DB\\FourClassesWithNSRmixed1519",
                                                                labels='inferred',label_mode="int",
                                                                class_names=['AF','NSR','PAC','PVC'],color_mode='rgb',
                                                                batch_size=batch_size,image_size=(IMG_SIZE,IMG_SIZE), #reshapeauto
                                                                shuffle=True,seed=123,validation_split=0.4,subset="validation")

Found 2421 files belonging to 4 classes.
Using 968 files for validation.
CPU times: total: 234 ms
Wall time: 235 ms


In [7]:
class_names = ds_train.class_names
print(class_names)

['AF', 'NSR', 'PAC', 'PVC']


In [8]:
%%time
val_batches = tf.data.experimental.cardinality(ds_validate)
test_dataset = ds_validate.take(300)
validation_dataset = ds_validate.take(300)
print('Number of validation batches: %d' % tf.data.experimental.cardinality(validation_dataset))
print('Number of test batches: %d' % tf.data.experimental.cardinality(test_dataset))


AUTOTUNE = tf.data.AUTOTUNE
train_dataset = ds_train.cache().shuffle(100).prefetch(buffer_size=AUTOTUNE)
val_dataset = ds_validate.cache().shuffle(100).prefetch(buffer_size=AUTOTUNE)

Number of validation batches: 10
Number of test batches: 10
CPU times: total: 0 ns
Wall time: 23 ms


tf.data.experimental.cardinality():
This function determines the cardinality of a dataset, i.e., the number of elements in the dataset. It helps us analyze the size and structure of our datasets.

Dataset Preparation Steps:

val_batches = tf.data.experimental.cardinality(ds_validate): We calculate the cardinality of the validation dataset ds_validate. The variable val_batches holds the number of batches in the validation dataset.

test_dataset = ds_validate.take(300): We create a new dataset test_dataset by taking the first 300 elements from ds_validate. This dataset will be used for testing the trained model.

validation_dataset = ds_validate.take(300): We create another dataset validation_dataset by also taking the first 300 elements from ds_validate. This dataset will be used for validation during model training.

AUTOTUNE = tf.data.AUTOTUNE: The AUTOTUNE value is used as a buffer size for prefetching data, allowing TensorFlow to optimize the data loading process automatically.

train_dataset = ds_train.cache().shuffle(100).prefetch(buffer_size=AUTOTUNE): We prepare the training dataset ds_train by caching it in memory, shuffling it with a buffer of 100 elements, and prefetching it using the AUTOTUNE buffer size.

val_dataset = ds_validate.cache().shuffle(100).prefetch(buffer_size=AUTOTUNE): Similarly, we prepare the validation dataset ds_validate by caching it in memory, shuffling it with a buffer of 100 elements, and prefetching it using the AUTOTUNE buffer size.

The Significance of Dataset Preparation:

By determining the cardinality of the validation dataset and creating subsets for validation and testing, we ensure that these datasets are of the appropriate size for model evaluation without overwhelming the memory.

Caching and prefetching the datasets allow us to optimize data loading and minimize the time spent on data I/O during model training, significantly speeding up the overall training process.

As we optimize the dataset preparation and profiling execution time, our data-driven journey continues to accelerate, unlocking the full potential of Python, TensorFlow, and GPU acceleration. Witness the remarkable progress as we delve deeper into the world of deep learning and ECG record analysis.

In [9]:
data_augmentation = tf.keras.Sequential([tf.keras.layers.experimental.preprocessing.RandomZoom(.5,.5),])

#### Data Augmentation for Enhanced Deep Learning
In our ongoing quest for improved model performance and generalization, we introduce a powerful technique called data augmentation. By utilizing data augmentation, we can increase the diversity of our training dataset and enhance the robustness of our deep learning model.

##### Data Augmentation with TensorFlow and Keras:
TensorFlow and Keras offer a wide array of data augmentation techniques to enrich the training dataset. Data augmentation involves applying various transformations to the images, such as rotation, translation, zooming, flipping, and more. These transformations introduce slight variations to the original images, creating new training samples that capture different aspects of the data distribution.

tf.keras.layers.experimental.preprocessing.RandomZoom:
The RandomZoom layer from TensorFlow's experimental preprocessing module allows us to apply random zooming to the images during training. The RandomZoom layer takes two arguments:

.5: The zoom factor range, where 0.5 represents a zooming range of 50%. This means that the images can be zoomed in or out by up to 50%.

.5: The height factor range, similar to the width factor range. In this case, we also set it to 0.5, representing a height zooming range of 50%.

The Power of Data Augmentation:
Data augmentation is an essential technique for mitigating overfitting and enhancing model generalization. By introducing variations to the training dataset through data augmentation, the model learns to be more robust to different patterns and image distortions that it may encounter during real-world scenarios.

By employing data augmentation alongside deep learning, we harness the full potential of our Python program and TensorFlow integration. Our model gains the ability to generalize better, improving its performance on unseen data and increasing its real-world applicability.

In [10]:
preprocess_input = tf.keras.applications.efficientnet.preprocess_input
rescale = tf.keras.layers.Rescaling(1./223.5, offset=-1)

#### Preprocessing Input and Rescaling for EfficientNet
In our ongoing effort to enhance the performance of our deep learning model, we introduce two crucial components: preprocess_input and Rescaling. These components play a critical role in preparing the input data for the EfficientNet model, ensuring that it operates effectively and efficiently.

preprocess_input for EfficientNet:
The preprocess_input function from tf.keras.applications.efficientnet is used to preprocess the input data before feeding it into the EfficientNet model. EfficientNet, like many pre-trained models, requires input data to be preprocessed in a specific manner to ensure optimal performance.

Rescaling Layer:
The Rescaling layer from TensorFlow's tf.keras.layers module is used to rescale the input data. In this case, the input data is rescaled using a scale factor of 1/223.5 and an offset of -1. This normalization ensures that the input data falls within a specific range, typically between 0 and 1, which is often beneficial for neural network models.

The Significance of Preprocessing and Rescaling:

Preprocessing the input data using the preprocess_input function ensures that the data is formatted in a manner compatible with the EfficientNet model's requirements. This step might involve resizing the images, converting color formats, and normalizing pixel values to a specific range.

The Rescaling layer is crucial for normalizing the input data to a standardized range. Normalizing the data helps the model converge faster during training, as it reduces the impact of large variations in pixel values.

EfficientNet:
EfficientNet is a highly efficient and accurate deep learning model that has gained popularity due to its excellent performance on various tasks. By integrating EfficientNet into our project and preparing the input data properly, we ensure that our model operates at its full potential.

In [11]:
# Create the base model from the pre-trained model EFB0
base_model = tf.keras.applications.EfficientNetB0(input_shape=(224,224,3),
                                                  include_top=False,
                                                  weights='imagenet',
                                                  classifier_activation='softmax')

#### Creating the Base Model with EfficientNetB0
As we venture into building our deep learning model, we lay a strong foundation by creating the base model using EfficientNetB0. EfficientNetB0 is a pre-trained deep learning model that has demonstrated excellent performance in various computer vision tasks, making it a promising candidate for our ECG record analysis.

EfficientNetB0:
EfficientNet is a family of pre-trained convolutional neural network models developed by Google. EfficientNetB0 is the smallest variant in the family, designed to strike a balance between efficiency and performance. Despite its compact size, EfficientNetB0 delivers impressive results across various image recognition tasks.

input_shape=(224, 224, 3):
We specify the input shape of the images that will be fed into the EfficientNetB0 model. The input shape consists of three components: width, height, and the number of channels. In this case, we set the input shape to (224, 224, 3), indicating that the images have a resolution of 224x224 pixels and three color channels (RGB).

include_top=False:
By setting include_top to False, we exclude the fully connected layers (top) of the EfficientNetB0 model. We do this because we intend to add our custom top layers that are tailored to our specific task (ECG record analysis).

weights='imagenet':
The weights parameter specifies the weight initialization of the model. By setting it to 'imagenet', we initialize the model with pre-trained weights on the ImageNet dataset. This initialization provides a head start in training, as the model has already learned meaningful representations from a large and diverse image dataset.

classifier_activation='softmax':
The classifier_activation parameter determines the activation function for the classification layer (output layer). In this case, we set it to 'softmax', which is suitable for multi-class classification tasks like our ECG record analysis, where we have multiple classes to predict.

The Importance of EfficientNetB0 as a Base Model:
By leveraging EfficientNetB0 as the base model, we can harness its powerful feature extraction capabilities. This significantly simplifies the process of building a deep learning model for ECG record analysis, as we can focus on customizing the top layers to suit our classification task.

In [12]:
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

(100, 7, 7, 1280)


#### Extracting Features using EfficientNetB0
As we embark on our deep learning journey, we take a significant step forward by using EfficientNetB0 to extract features from the input image batch. These features serve as essential representations of the images and are critical for building a robust and accurate ECG classification model.

next(iter(train_dataset)):
The next(iter(train_dataset)) expression retrieves the next batch of images and labels from the train_dataset. We use the Python iter() function to create an iterator from the train_dataset, and next() fetches the next batch.

base_model(image_batch):
We pass the image_batch through the base_model, which is the EfficientNetB0 model that we created earlier. The model extracts features from the input images.

print(feature_batch.shape):
The print() statement outputs the shape of the feature_batch, allowing us to inspect the dimensions of the extracted features.

Extracting Features with EfficientNetB0:
EfficientNetB0 is a powerful deep learning model that excels in feature extraction. By passing the image_batch through EfficientNetB0, we obtain a feature_batch, where each element in the batch represents the extracted features of a corresponding image.

The Significance of Feature Extraction:
Feature extraction is a crucial step in deep learning, as it enables the model to identify and represent relevant patterns and structures in the data. EfficientNetB0's ability to extract high-level features from images allows our ECG record analysis model to capture meaningful information that is essential for accurate classification.

Understanding the feature_batch Shape:
The feature_batch.shape output provides us with information about the shape of the extracted features. The shape typically consists of four dimensions, representing the batch size, spatial dimensions, and number of channels of the features.

For example, if the output is (32, 7, 7, 1280), it indicates that the feature_batch contains 32 images (batch size) and each image is represented by a feature map of size 7x7 with 1280 channels.

In [13]:
base_model.trainable = False #Freeze the convolutional base

#### Freezing the Convolutional Base

In our quest for effective transfer learning and model optimization, we take a crucial step by freezing the convolutional base of EfficientNetB0. By doing so, we prevent the weights of the base model from being updated during training, allowing us to focus on training only the custom top layers for our specific classification task.

Freezing the Convolutional Base:
When we set base_model.trainable to False, we freeze the convolutional base of EfficientNetB0. Freezing means that the weights of the base model's layers will remain fixed, and they will not be updated during the training process. This applies to all layers in the base model, including the feature extraction layers.

The Importance of Freezing the Convolutional Base:
Freezing the convolutional base has several advantages:

Transfer Learning: EfficientNetB0 is a pre-trained model that has already learned useful features from a vast dataset (ImageNet). By freezing the convolutional base, we preserve these valuable pre-trained representations, which can greatly benefit our ECG record analysis task.

Reduced Training Time: The majority of the model's parameters lie in the convolutional base. By freezing these layers, we significantly reduce the number of trainable parameters, leading to faster training times and lower memory requirements.

Preventing Overfitting: Freezing the convolutional base helps prevent overfitting, especially when working with limited training data. Overfitting occurs when a model memorizes the training data, leading to poor generalization on new, unseen data. By freezing the base, we ensure that the model focuses on learning task-specific patterns rather than fitting to the training examples.

Custom Top Layers: With the convolutional base frozen, we can now add custom top layers (such as fully connected or dense layers) on top of the EfficientNetB0 base. These custom layers will be responsible for learning task-specific representations and making predictions for our ECG record analysis.

In [14]:
# Let's take a look at the base model architecture
base_model.summary()
len(base_model.layers)

Model: "efficientnetb0"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling_1 (Rescaling)     (None, 224, 224, 3)          0         ['input_1[0][0]']             
                                                                                                  
 normalization (Normalizati  (None, 224, 224, 3)          7         ['rescaling_1[0][0]']         
 on)                                                                                              
                                                                                                  
 rescaling_2 (Rescaling)     (None, 224, 224, 3)          0         ['normalization[0

 y)                                                                  'block2a_se_expand[0][0]']   
                                                                                                  
 block2a_project_conv (Conv  (None, 56, 56, 24)           2304      ['block2a_se_excite[0][0]']   
 2D)                                                                                              
                                                                                                  
 block2a_project_bn (BatchN  (None, 56, 56, 24)           96        ['block2a_project_conv[0][0]']
 ormalization)                                                                                    
                                                                                                  
 block2b_expand_conv (Conv2  (None, 56, 56, 144)          3456      ['block2a_project_bn[0][0]']  
 D)                                                                                               
          

                                                                                                  
 block3a_project_conv (Conv  (None, 28, 28, 40)           5760      ['block3a_se_excite[0][0]']   
 2D)                                                                                              
                                                                                                  
 block3a_project_bn (BatchN  (None, 28, 28, 40)           160       ['block3a_project_conv[0][0]']
 ormalization)                                                                                    
                                                                                                  
 block3b_expand_conv (Conv2  (None, 28, 28, 240)          9600      ['block3a_project_bn[0][0]']  
 D)                                                                                               
                                                                                                  
 block3b_e

 block4a_project_conv (Conv  (None, 14, 14, 80)           19200     ['block4a_se_excite[0][0]']   
 2D)                                                                                              
                                                                                                  
 block4a_project_bn (BatchN  (None, 14, 14, 80)           320       ['block4a_project_conv[0][0]']
 ormalization)                                                                                    
                                                                                                  
 block4b_expand_conv (Conv2  (None, 14, 14, 480)          38400     ['block4a_project_bn[0][0]']  
 D)                                                                                               
                                                                                                  
 block4b_expand_bn (BatchNo  (None, 14, 14, 480)          1920      ['block4b_expand_conv[0][0]'] 
 rmalizati

 ormalization)                                                                                    
                                                                                                  
 block4c_drop (Dropout)      (None, 14, 14, 80)           0         ['block4c_project_bn[0][0]']  
                                                                                                  
 block4c_add (Add)           (None, 14, 14, 80)           0         ['block4c_drop[0][0]',        
                                                                     'block4b_add[0][0]']         
                                                                                                  
 block5a_expand_conv (Conv2  (None, 14, 14, 480)          38400     ['block4c_add[0][0]']         
 D)                                                                                               
                                                                                                  
 block5a_e

 block5b_add (Add)           (None, 14, 14, 112)          0         ['block5b_drop[0][0]',        
                                                                     'block5a_project_bn[0][0]']  
                                                                                                  
 block5c_expand_conv (Conv2  (None, 14, 14, 672)          75264     ['block5b_add[0][0]']         
 D)                                                                                               
                                                                                                  
 block5c_expand_bn (BatchNo  (None, 14, 14, 672)          2688      ['block5c_expand_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 block5c_expand_activation   (None, 14, 14, 672)          0         ['block5c_expand_bn[0][0]']   
 (Activati

 ormalization)                                                                                    
                                                                                                  
 block6b_expand_conv (Conv2  (None, 7, 7, 1152)           221184    ['block6a_project_bn[0][0]']  
 D)                                                                                               
                                                                                                  
 block6b_expand_bn (BatchNo  (None, 7, 7, 1152)           4608      ['block6b_expand_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 block6b_expand_activation   (None, 7, 7, 1152)           0         ['block6b_expand_bn[0][0]']   
 (Activation)                                                                                     
          

 block6c_add (Add)           (None, 7, 7, 192)            0         ['block6c_drop[0][0]',        
                                                                     'block6b_add[0][0]']         
                                                                                                  
 block6d_expand_conv (Conv2  (None, 7, 7, 1152)           221184    ['block6c_add[0][0]']         
 D)                                                                                               
                                                                                                  
 block6d_expand_bn (BatchNo  (None, 7, 7, 1152)           4608      ['block6d_expand_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 block6d_expand_activation   (None, 7, 7, 1152)           0         ['block6d_expand_bn[0][0]']   
 (Activati

                                                                                                  
 top_bn (BatchNormalization  (None, 7, 7, 1280)           5120      ['top_conv[0][0]']            
 )                                                                                                
                                                                                                  
 top_activation (Activation  (None, 7, 7, 1280)           0         ['top_bn[0][0]']              
 )                                                                                                
                                                                                                  
Total params: 4049571 (15.45 MB)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 4049571 (15.45 MB)
__________________________________________________________________________________________________


238

In [15]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

(100, 1280)


#### Global Average Pooling for Feature Aggregation
In our endeavor to process the extracted features effectively, we introduce Global Average Pooling 2D (GAP), a crucial operation for feature aggregation. By employing GAP, we transform the spatial dimensions of the feature maps into a single value per channel, condensing the information into a compact representation.

tf.keras.layers.GlobalAveragePooling2D():
The GlobalAveragePooling2D layer is employed to perform Global Average Pooling on the input feature maps. This operation is specifically designed for 2D data (images) and is often used after the convolutional layers to aggregate spatial information.

Feature Aggregation with Global Average Pooling:
When we apply GlobalAveragePooling2D to the feature_batch, it reduces the spatial dimensions (width and height) of each feature map to a single value per channel. This aggregation is performed independently for each channel in the feature maps.

The Significance of Feature Aggregation:
Feature aggregation with Global Average Pooling is beneficial for several reasons:

Dimension Reduction: Global Average Pooling reduces the spatial dimensions of the feature maps, effectively compressing the information into a more concise representation. This helps in reducing the computational complexity and memory requirements in subsequent layers.

Translation Invariance: Global Average Pooling makes the model more translation-invariant. By averaging features across the spatial dimensions, the model becomes less sensitive to the exact location of the features within an image. This can lead to improved generalization and robustness.

Global Context: Aggregating features globally enables the model to consider the entire input image when making predictions. This global context can be especially useful in tasks where important features may be spread across the entire image.

Understanding the feature_batch_average Shape:
The feature_batch_average.shape output represents the shape of the feature maps after Global Average Pooling is applied. Since GAP reduces the spatial dimensions to a single value per channel, the resulting shape will typically be (batch_size, num_channels).

For example, if the output is (32, 1280), it indicates that the feature_batch_average has 32 elements (images in the batch), and each image's features are represented by a vector of length 1280 (one value per channel).

In [16]:
number_of_classes = 4
inputs = tf.keras.Input(shape=(224,224, 3))
x = data_augmentation(inputs)
x = preprocess_input(inputs)
x = base_model(x, training=False)
      # call it on the given tensor
x = keras.layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.4)(x)
initializer = tf.keras.initializers.GlorotUniform(seed=123)
activation = None  #tf.keras.activations.softmax
outputs =outputs = keras.layers.Dense(number_of_classes,
                                      activation=activation,name="predictions")(x) 
efficientNet = tf.keras.Model(inputs, outputs)

By building on the foundation of EfficientNetB0, data augmentation, and Global Average Pooling, we create a custom model tailored to our specific ECG classification task.

The Model Building Process:
The model is constructed in a step-by-step manner:

Input Layer: We define the input layer with the desired input shape (224, 224, 3) to accommodate the images in our ECG record dataset.

Data Augmentation: The input images undergo data augmentation using the previously defined data_augmentation layer. Data augmentation introduces random transformations to the images, increasing the diversity of the training dataset and improving the model's ability to generalize.

Preprocess Input: The images are preprocessed using the preprocess_input function, ensuring they are formatted appropriately for EfficientNetB0.

Feature Extraction: The preprocessed images are passed through the EfficientNetB0 base model, and features are extracted using the previously defined base_model.

Global Average Pooling: Global Average Pooling is applied to the extracted features, aggregating spatial information into a compact representation.

Batch Normalization: Batch Normalization is added to improve the convergence speed and stability of the training process.

Dropout: Dropout regularization is implemented with a dropout rate of 0.4 to prevent overfitting during training.

Dense Layer (Classification Layer): A dense layer (fully connected layer) with number_of_classes neurons is added to perform the final classification. Since this is a multi-class classification task, we omit specifying an activation function at this stage.

The EfficientNet-Based ECG Classification Model: We combine the input and output layers to create the final model named efficientNet.

The Significance of the Model Architecture:
By customizing the EfficientNetB0 base model and adding additional layers, we create a cardiac arrhythmias classification model that is specifically tailored to our classification task. The model is designed to extract relevant features from ECG images, aggregate them through Global Average Pooling, and make accurate predictions for the multiple classes in our dataset.

In [17]:
base_learning_rate = 0.01
efficientNet.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), # default from_logits=False
              metrics=[keras.metrics.SparseCategoricalAccuracy()])

#### Compiling and Configuring the EfficientNet Model
With the ECG record analysis model architecture in place, we move forward to compile and configure the model. This step involves specifying the optimizer, loss function, and evaluation metrics, enabling us to begin the training process.

Model Compilation and Configuration:
The compilation step involves defining various components essential for model training:

Optimizer: We use the Adam optimizer to train the model. Adam is an efficient optimization algorithm that adapts the learning rate dynamically during training, making it well-suited for deep learning tasks.

Learning Rate: We specify the initial learning rate for the Adam optimizer. The base_learning_rate is set to 0.01, which determines how much the model's weights are updated during each training step. The learning rate can be tuned to control the rate of convergence and model performance.

Loss Function: For multi-class classification tasks with integer labels (like our ECG record analysis), we use SparseCategoricalCrossentropy as the loss function. This function is appropriate when the target labels are provided as integers, and the model's output contains logits (unnormalized log probabilities). Setting from_logits=True indicates that the model outputs logits, which is usually the case for multi-class classification tasks.

Metrics: We use SparseCategoricalAccuracy as the evaluation metric. This metric calculates the accuracy of the model predictions when the target labels are provided as integers (sparse).

The Significance of Model Compilation:
Compiling the model is a crucial step in preparing it for training. By specifying the optimizer, loss function, and metrics, we configure the model to optimize its parameters, compute the loss during training, and evaluate its performance on the validation dataset.

Training Begins:
Once the model is compiled and configured, we are all set to begin the training process. During training, the model will iteratively update its weights using the specified optimizer, minimizing the loss function, and improving its performance on the ECG record analysis task.

In [ ]:
%%time
first_hist=efficientNet.fit(train_dataset, validation_data=val_dataset, batch_size=100, epochs=100)


Epoch 1/100
15/15 [==============================] - 87s 5s/step - loss: 1.2128 - sparse_categorical_accuracy: 0.5547 - val_loss: 0.9373 - val_sparse_categorical_accuracy: 0.6777
Epoch 2/100
15/15 [==============================] - 74s 5s/step - loss: 0.5291 - sparse_categorical_accuracy: 0.8156 - val_loss: 0.8074 - val_sparse_categorical_accuracy: 0.7448
Epoch 3/100
15/15 [==============================] - 73s 5s/step - loss: 0.3981 - sparse_categorical_accuracy: 0.8541 - val_loss: 0.7310 - val_sparse_categorical_accuracy: 0.7851
Epoch 4/100
15/15 [==============================] - 79s 5s/step - loss: 0.3392 - sparse_categorical_accuracy: 0.8844 - val_loss: 0.6721 - val_sparse_categorical_accuracy: 0.8233
Epoch 5/100
15/15 [==============================] - ETA: 0s - loss: 0.2787 - sparse_categorical_accuracy: 0.9071

#### Training the EfficientNet Model

With the model compiled and configured, we proceed to the training phase, where the EfficientNet-based ECG record analysis model learns from the training dataset. Training involves iteratively updating the model's weights to minimize the loss function and improve its ability to make accurate predictions.

Training the Model:

efficientNet.fit: The fit method is called on the efficientNet model to initiate the training process. This method takes the following arguments:

train_dataset: The training dataset, which contains the ECG record images and their corresponding labels. The model will learn from this dataset to make accurate predictions.

validation_data: The validation dataset, which is used to evaluate the model's performance during training. It contains images and labels separate from the training dataset and helps us monitor the model's ability to generalize to unseen data.

batch_size: The number of samples in each batch during training. In this case, we use a batch size of 100, meaning the model will update its weights after processing each batch of 100 images.

epochs: The number of times the model will iterate over the entire training dataset during training. In this case, we set epochs=100, which means the model will go through the entire training dataset 100 times.

Monitoring Training Time:
The %%time magic command is used to measure the training time. This command calculates and displays the time taken to complete the training process.

The Training Process:
During training, the model processes batches of training data, computes the loss function, and updates its weights using the Adam optimizer. The model's performance on the validation dataset is evaluated after each epoch, allowing us to monitor its progress and identify possible overfitting or underfitting.

In [ ]:
efficientNet.save('ecg_trial_model.h5')

#### Saving the Trained EfficientNet Model

Congratulations on completing the training phase of the EfficientNet-based ECG record analysis model! The progress you've made so far is truly commendable. Now, let's take a moment to preserve this well-trained model for future use and further exploration.

Saving the Model:
The save method is called on the efficientNet model to save it to a file. We provide the filename as "ecg_trial_model.h5". The ".h5" extension is a common choice for saving models using the Hierarchical Data Format (HDF5), which is a binary data format commonly used in machine learning.

Model Preservation:
By saving the model to a file, we ensure that the trained weights, architecture, and configuration of the model are stored for future use. This allows us to easily reload the model at a later time without needing to retrain it, which can be especially useful for tasks such as evaluation, deployment, and transfer learning.

In [ ]:
tf.saved_model.save(efficientNet, "ecgmodel_saved")

#### Saving the Trained EfficientNet Model as a SavedModel
In addition to saving the model in the HDF5 format, TensorFlow provides another convenient way to save models, known as the SavedModel format. This format allows for a more comprehensive preservation of the model, including its architecture, weights, and computation graph, making it easy to deploy the model across different platforms and environments.

Saving as a SavedModel:
The tf.saved_model.save function is used to save the model as a SavedModel. We pass the efficientNet model object as the first argument and specify the directory where the SavedModel will be saved. In this case, we provide the directory name as "ecgmodel_saved".

Advantages of the SavedModel Format:
The SavedModel format offers several benefits:

Comprehensive Preservation: The SavedModel format preserves not only the model's architecture and weights but also the computation graph, custom layers, and optimizer states. This makes it easier to reload and deploy the model without losing any essential information.

Platform-Agnostic: The SavedModel format is platform-agnostic, meaning it can be used across different platforms, languages, and TensorFlow versions. This portability is particularly useful when deploying models to production environments or sharing them with others.

Serving with TensorFlow Serving: The SavedModel format is the standard for deploying models with TensorFlow Serving, a dedicated system for deploying machine learning models in production.

In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(efficientNet)
efficient_tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(efficient_tflite_model)

#### Converting the Trained EfficientNet Model to TensorFlow Lite

With the trained EfficientNet model at our disposal, we can take a significant step towards optimizing and deploying the model on resource-constrained devices. TensorFlow Lite (TFLite) is a framework designed for running machine learning models efficiently on edge devices, mobile devices, and microcontrollers.

Converting to TensorFlow Lite:
To convert the trained EfficientNet model to TensorFlow Lite, we use the tf.lite.TFLiteConverter.from_keras_model function. This converter takes the efficientNet model as input and performs the necessary transformations to optimize it for deployment on edge devices.

Saving the TensorFlow Lite Model:
After converting the model, we save it to a file with the extension ".tflite". The model is written to the file in binary format using the write method.

##### The Benefits of TensorFlow Lite:
TensorFlow Lite offers several advantages:

Efficiency: TensorFlow Lite is designed for running models efficiently on devices with limited computational resources, making it well-suited for edge devices and mobile applications.

Accelerated Inference: TFLite supports hardware acceleration (e.g., through the use of Tensor Processing Units, GPU acceleration, or Neural Processing Units), leading to faster inference times on compatible devices.

Reduced Model Size: Models converted to TensorFlow Lite are often smaller in size compared to their original counterparts, making them easier to deploy and distribute.

On-Device Inference: TensorFlow Lite enables on-device inference, meaning the model can make predictions directly on the device without relying on a cloud or server.

In [ ]:
acc = first_hist.history['sparse_categorical_accuracy']
val_acc = first_hist.history['val_sparse_categorical_accuracy']

loss = first_hist.history['loss']
val_loss = first_hist.history['val_loss']

plt.figure(figsize=(8, 8))
plt.style.use('_classic_test_patch')
plt.grid(True)

plt.subplot(2, 1, 1)
plt.style.use('_classic_test_patch')
plt.grid(True)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy with EfficientNet')

plt.subplot(2, 1, 2)
plt.style.use('_classic_test_patch')
plt.grid(True)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss with EfficientNet')
plt.xlabel('epoch')
plt.show()

#### Visualization of Training and Validation Metrics
Visualizing the training and validation metrics helps us gain insights into the model's performance during the training process. By plotting the accuracy and loss values over the epochs, we can observe the model's learning progress and identify possible overfitting or underfitting.

Plotting Training and Validation Metrics:
In this code snippet, we use matplotlib to create two subplots, each displaying the training and validation metrics for accuracy and loss.

Training and Validation Accuracy Plot: The first subplot displays the training and validation accuracy over the epochs. The training accuracy is plotted as a line graph with the label "Training Accuracy," while the validation accuracy is represented by another line graph with the label "Validation Accuracy."

Training and Validation Loss Plot: The second subplot shows the training and validation loss over the epochs. The training loss is depicted as a line graph with the label "Training Loss," and the validation loss is displayed as another line graph with the label "Validation Loss."

##### Interpreting the Plots:

For the accuracy plot, higher values indicate better performance. Ideally, both training and validation accuracy should increase with each epoch, but if validation accuracy starts to plateau or decrease while training accuracy continues to increase, it may be a sign of overfitting.

For the loss plot, lower values indicate better performance. Both training and validation loss should decrease with each epoch. If validation loss starts to increase while training loss continues to decrease, it may indicate overfitting.

By analyzing these plots, we can make informed decisions about model optimization and generalization, ensuring the trained model performs well on unseen data.

In [ ]:
%%time
loss, accuracy= efficientNet.evaluate(test_dataset)
print('Test accuracy :', accuracy)


#### Evaluating the Trained EfficientNet Model on the Test Dataset
To gauge the model's performance on unseen data, we evaluate the trained EfficientNet model on the test dataset. By computing the loss and accuracy metrics on the test dataset, we can ascertain how well the model generalizes to real-world examples.

Model Evaluation:
In this code snippet, we use the evaluate method to assess the model's performance on the test dataset multiple times.

Evaluating Test Accuracy:
For each evaluation, the model's loss and accuracy on the test dataset are computed. The resulting accuracy indicates the proportion of correctly classified samples in the test dataset.

Interpreting Test Accuracy:
The test accuracy provides valuable insights into how well the model generalizes to unseen data. A high test accuracy indicates that the model is performing well on the test dataset and can make accurate predictions on real-world examples. On the other hand, a low test accuracy may suggest overfitting or limitations in generalization.

In [ ]:
%%time
ds_validate=tf.keras.preprocessing.image_dataset_from_directory("D:\\ECG DB\\FourClasseswith 20percent mixed",
                                                                labels='inferred',label_mode="int",
                                                                class_names=['AF','NSR','PAC','PVC'],color_mode='rgb',
                                                                batch_size=548,image_size=(IMG_SIZE,IMG_SIZE), #reshapeauto
                                                                shuffle=True,seed=123,validation_split=0.4,subset="validation")

test_dataset = ds_validate.take(548)
validation_dataset = ds_validate.skip(400)
print('Number of validation batches: %d' % tf.data.experimental.cardinality(validation_dataset))
print('Number of test batches: %d' % tf.data.experimental.cardinality(test_dataset))


In [ ]:
batch_size=548
image_batch, label_batch = test_dataset.as_numpy_iterator().next()
predictions = efficientNet.predict(image_batch)
predictions.shape

In [ ]:
predicted_classes=[]
for i in range(548):
    predicted_class=np.argmax(predictions[i])
    predicted_classes.append(predicted_class)

In [ ]:
# Confusion Matrix
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

y_true=label_batch
y_pred=np.array(predicted_classes)
confusion_matrix(y_true, y_pred)

data = {'y_Actual': y_true,
        'y_Predicted': y_pred
        }

df = pd.DataFrame(data, columns=['y_Actual','y_Predicted'])
confusion_matrix = pd.crosstab(df['y_Actual'], df['y_Predicted'], rownames=['Actual'], colnames=['Predicted'])
print (confusion_matrix)
sn.heatmap(confusion_matrix, annot=True)
plt.show()

In [ ]:
confusion_matrix

In [ ]:
def AF_SandS (conf_matrix):
    TP= conf_matrix[0][0]
    TN= (conf_matrix[1][1]+conf_matrix[2][2]+conf_matrix[3][3]+
               conf_matrix[1][2]+conf_matrix[1][3]+conf_matrix[2][1]+conf_matrix[2][3]+conf_matrix[3][1]+conf_matrix[3][2])
    FP= conf_matrix[0][1]+conf_matrix[0][2]+conf_matrix[0][3]
    FN= conf_matrix[1][0]+conf_matrix[2][0]+conf_matrix[3][0]    
    accuracy = (float (TP+TN) / float(TP + TN + FP + FN))
    misclassification = 1- accuracy
    sensitivity = (TP / float(TP + FN))
    specificity = (TN / float(TN + FP))
    
    print('-'*50)
    print('AF')
    print(f'Accuracy: {(accuracy)}') 
    print(f'Mis-Classification: {round(misclassification,2)}') 
    print(f'Sensitivity: {(sensitivity)}') 
    print(f'Specificity: {(specificity)}') 

    
def NSR_SandS (conf_matrix):
    TP= conf_matrix[1][1]
    TN= (conf_matrix[0][0]+conf_matrix[2][2]+conf_matrix[3][3]+
               conf_matrix[0][2]+conf_matrix[0][3]+conf_matrix[2][0]+conf_matrix[2][3]+conf_matrix[3][0]+conf_matrix[3][2])
    FP= conf_matrix[1][0]+conf_matrix[1][2]+conf_matrix[1][3]
    FN= conf_matrix[0][1]+conf_matrix[2][1]+conf_matrix[3][1]    
    #accuracy = (float (TP+TN) / float(TP + TN + FP + FN))
    misclassification = 1- accuracy
    sensitivity = (TP / float(TP + FN))
    specificity = (TN / float(TN + FP))
    
    print('-'*50)
    print('NSR')
    print(f'Accuracy: {(accuracy)}') 
    print(f'Mis-Classification: {round(misclassification,2)}') 
    print(f'Sensitivity: {(sensitivity)}') 
    print(f'Specificity: {(specificity)}') 

In [ ]:
def PAC_SandS (conf_matrix):
    TP= conf_matrix[2][2]
    TN= (conf_matrix[0][0]+conf_matrix[1][1]+conf_matrix[3][3]+
               conf_matrix[0][1]+conf_matrix[0][3]+conf_matrix[1][0]+conf_matrix[1][3]+conf_matrix[3][0]+conf_matrix[3][1])
    FP= conf_matrix[2][0]+conf_matrix[2][1]+conf_matrix[2][3]
    FN= conf_matrix[0][2]+conf_matrix[1][2]+conf_matrix[3][2]    
    accuracy = (float (TP+TN) / float(TP + TN + FP + FN))
    misclassification = 1- accuracy
    sensitivity = (TP / float(TP + FN))
    specificity = (TN / float(TN + FP))
    
    print('-'*50)
    print('PAC')
    print(f'Accuracy: {(accuracy)}') 
    print(f'Mis-Classification: {round(misclassification,2)}') 
    print(f'Sensitivity: {(sensitivity)}') 
    print(f'Specificity: {(specificity)}') 
    
    
def PVC_SandS (conf_matrix):
    TP= conf_matrix[3][3]
    TN= (conf_matrix[0][0]+conf_matrix[1][1]+conf_matrix[2][2]+
               conf_matrix[0][1]+conf_matrix[0][2]+conf_matrix[1][0]+conf_matrix[1][2]+conf_matrix[2][0]+conf_matrix[2][1])
    FP= conf_matrix[3][0]+conf_matrix[3][1]+conf_matrix[3][2]
    FN= conf_matrix[0][3]+conf_matrix[1][3]+conf_matrix[2][3]    
    accuracy = (float (TP+TN) / float(TP + TN + FP + FN))
    misclassification = 1- accuracy
    sensitivity = (TP / float(TP + FN))
    specificity = (TN / float(TN + FP))
    
    print('-'*50)
    print('PVC')
    print(f'Accuracy: {(accuracy)}') 
    print(f'Mis-Classification: {round(misclassification,2)}') 
    print(f'Sensitivity: {(sensitivity)}') 
    print(f'Specificity: {(specificity)}') 

In [ ]:
AF_SandS(confusion_matrix)
NSR_SandS(confusion_matrix)
PAC_SandS(confusion_matrix)
PVC_SandS(confusion_matrix)

In [ ]:
from sklearn.metrics import classification_report
y_true=label_batch
y_pred=np.array(predicted_classes)
target_names = ['AF', 'NSR', 'PAC','PVC']
print(classification_report(y_true, y_pred, target_names=target_names))


In [ ]:
mismatched=np.where(np.array(predicted_classes)!=label_batch)[0]
mismatched


In [ ]:
plt.suptitle("REAL is"+target_names[label_batch[mismatched[3]]])
plt.title("Predicted as"+target_names[predicted_classes[mismatched[3]]])
plt.axis("off")
plt.imshow(image_batch[mismatched[3]].astype("uint8"))
#plt.savefig(str(0))

In [ ]:
test_dataset=keras.utils.image_dataset_from_directory(directory='D:\\ECG DB\\TTV twentypercent\\Test',
                                                    labels='inferred',
                                                    label_mode='int',
                                                    class_names=['AF','NSR','PAC','PVC'],
                                                    batch_size=561,shuffle=True,seed=123,
                                                    image_size=(224, 224))
image_batch, label_batch = test_dataset.as_numpy_iterator().next()
predictions = efficientNet.predict(image_batch)
predictions.shape
predicted_classes=[]
for i in range(561):
    predicted_class=np.argmax(predictions[i])
    predicted_classes.append(predicted_class)

In [ ]:
# Confusion Matrix
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

y_true=label_batch
y_pred=np.array(predicted_classes)
confusion_matrix(y_true, y_pred)

data = {'y_Actual': y_true,
        'y_Predicted': y_pred
        }

df = pd.DataFrame(data, columns=['y_Actual','y_Predicted'])
confusion_matrix = pd.crosstab(df['y_Actual'], df['y_Predicted'], rownames=['Actual'], colnames=['Predicted'])
print (confusion_matrix)
sn.heatmap(confusion_matrix, annot=True)
plt.show()

In [ ]:
test_dataset=keras.utils.image_dataset_from_directory(directory='D:\\ECG DB\\PAC 11to15',
                                                    labels='inferred',
                                                    label_mode='int',
                                                    class_names=['PAC'],
                                                    batch_size=100,shuffle=True,seed=123,
                                                    image_size=(224, 224))
image_batch, label_batch = test_dataset.as_numpy_iterator().next()
predictions = efficientNet.predict(image_batch)
predictions.shape
predicted_classes=[]
for i in range(100):
    predicted_class=np.argmax(predictions[i])
    predicted_classes.append(predicted_class)

In [ ]:
# Confusion Matrix
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

y_true=label_batch
y_pred=np.array(predicted_classes)
confusion_matrix(y_true, y_pred)

data = {'y_Actual': y_true,
        'y_Predicted': y_pred
        }

df = pd.DataFrame(data, columns=['y_Actual','y_Predicted'])
confusion_matrix = pd.crosstab(df['y_Actual'], df['y_Predicted'], rownames=['Actual'], colnames=['Predicted'])
print (confusion_matrix)
sn.heatmap(confusion_matrix, annot=True)
plt.show()